# Setup

In [7]:
# !proxychains pip install langchain-tavily

In [8]:
from langfuse.callback import CallbackHandler

langfuse_handler = CallbackHandler(
  secret_key="",
  public_key="pk-lf-ba431a2c-da14-42c0-9e43-5af9e5f4d126",
  host="https://cloud.langfuse.com"
)

# chain.invoke({"input": "<user_input>"}, config={"callbacks": [langfuse_handler]})

In [9]:
from dotenv import load_dotenv
load_dotenv()

True

In [10]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-4o-mini", model_provider="openai")

In [11]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [12]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

In [40]:
from langchain_tavily import TavilySearch

search_tool = TavilySearch(max_results=2)

In [21]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

# tools

In [74]:
from langgraph.types import Command, interrupt
from langchain_core.tools import InjectedToolCallId, tool
from langchain_core.messages import ToolMessage

In [4]:
# @tool
# def human_assistance(query: str) -> str:
#     """Request assistance from a human."""
#     human_response = interrupt({"query": query})
#     return human_response["data"]


@tool
def human_assistance(
    name: str, birthday: str, tool_call_id: Annotated[str, InjectedToolCallId]
) -> str:
    """Request assistance from a human."""
    human_response = interrupt(
        {
            "question": "Is this correct?",
            "name": name,
            "birthday": birthday,
        },
    )
    # If the information is correct, update the state as-is.
    if human_response.get("correct", "").lower().startswith("y"):
        verified_name = name
        verified_birthday = birthday
        response = "Correct"
    # Otherwise, receive information from the human reviewer.
    else:
        verified_name = human_response.get("name", name)
        verified_birthday = human_response.get("birthday", birthday)
        response = f"Made a correction: {human_response}"

    # This time we explicitly update the state with a ToolMessage inside
    # the tool.
    state_update = {
        "name": verified_name,
        "birthday": verified_birthday,
        "messages": [ToolMessage(response, tool_call_id=tool_call_id)],
    }
    # We return a Command object in the tool to update our state.
    return Command(update=state_update)
    
tools = [search_tool, human_assistance]
llm_with_tools = llm.bind_tools(tools)

# graph

In [1]:
from typing import Annotated

from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

class State(TypedDict):
    messages: Annotated[list, add_messages]
    name: str
    birthday: str

def chatbot(state: State):
    message = llm_with_tools.invoke(state["messages"])
    assert len(message.tool_calls) <= 1
    return {"messages": [message]}

In [2]:
graph_builder = StateGraph(State)
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_edge(START, "chatbot")

tool_node = ToolNode(tools=tools)
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
# Any time a tool is called, we return to the chatbot to decide the next step
graph_builder.add_edge("tools", "chatbot")
graph = graph_builder.compile(checkpointer=memory)
graph

NameError: name 'tools' is not defined

# Usage

In [23]:
config = {"configurable": {"thread_id": "1"}}

In [24]:
user_input = "Hi there! My name is Will."

# The config is the **second positional argument** to stream() or invoke()!
events = graph.stream(
    {"messages": [{"role": "user", "content": user_input}]},
    config,
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Hi there! My name is Will.
================================== Ai Message ==================================

Hello, Will! How can I assist you today?


In [26]:
user_input = "Remember my name?"

# The config is the **second positional argument** to stream() or invoke()!
events = graph.stream(
    {"messages": [{"role": "user", "content": user_input}]},
    config,
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Remember my name?
================================== Ai Message ==================================

Yes, Will! I remember your name. How can I assist you today?


In [27]:
user_input = "Do you know anything about agentic programming?"

# The config is the **second positional argument** to stream() or invoke()!
events = graph.stream(
    {"messages": [{"role": "user", "content": user_input}]},
    config,
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Do you know anything about agentic programming?
================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_HWMo0SeoZ4dFS3czfZXvulSD)
 Call ID: call_HWMo0SeoZ4dFS3czfZXvulSD
  Args:
    query: agentic programming
    search_depth: advanced
================================= Tool Message =================================
Name: tavily_search

{"query": "agentic programming", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://en.wikipedia.org/wiki/Agent-oriented_programming", "title": "Agent-oriented programming - Wikipedia", "content": "Agent-oriented programming (AOP) is a programming paradigm where the construction of the software is centered on the concept of software agents.", "score": 0.8482825, "raw_content": null}, {"url": "https://www.riis.com/blog/introduction-to-agentic-programming-part-1", "title": "Introdu

In [36]:
snapshot = graph.get_state(config)
for message in snapshot.values["messages"]:
    print(message.pretty_print())

================================ Human Message =================================

Hi there! My name is Will.
None
================================== Ai Message ==================================

Hello, Will! How can I assist you today?
None
================================ Human Message =================================

Hi there! My name is Will.
None
================================== Ai Message ==================================

Hello again, Will! How can I help you today?
None
================================ Human Message =================================

Remember my name?
None
================================== Ai Message ==================================

Yes, Will! I remember your name. How can I assist you today?
None
================================ Human Message =================================

Do you know anything about agentic programming?
None
================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_HWMo0SeoZ

# Usage with human intruption

In [76]:
user_input = (
    "Can you look up when LangGraph was released? "
    "When you have the answer, use the human_assistance tool for review."
)
config = {"configurable": {"thread_id": "3"}}

events = graph.stream(
    {"messages": [{"role": "user", "content": user_input}]},
    config,
    stream_mode="values",
)
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================ Human Message =================================

Can you look up when LangGraph was released? When you have the answer, use the human_assistance tool for review.
================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_J0m26OjCImjrbOejEhg47lSC)
 Call ID: call_J0m26OjCImjrbOejEhg47lSC
  Args:
    query: LangGraph release date
================================= Tool Message =================================
Name: tavily_search

{"query": "LangGraph release date", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Releases · langchain-ai/langgraph - GitHub", "url": "https://github.com/langchain-ai/langgraph/releases", "content": "Releases · langchain-ai/langgraph GitHub Copilot Write better code with AI GitHub Advanced Security Find and fix vulnerabilities Code Search Find more, search less *   Why GitHub *   GitHub Advanced Security Enterprise-grade security features S

In [77]:
human_command = Command(
    resume={
        "name": "LangGraph",
        "birthday": "Jan 17, 2024",
    },
)

events = graph.stream(human_command, config, stream_mode="values")
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================== Ai Message ==================================
Tool Calls:
  human_assistance (call_RcZBN6iZP24ZjnbCpsxatNGj)
 Call ID: call_RcZBN6iZP24ZjnbCpsxatNGj
  Args:
    name: LangGraph
    birthday: 2023-10-10
================================= Tool Message =================================
Name: human_assistance

Made a correction: {'name': 'LangGraph', 'birthday': 'Jan 17, 2024'}
================================== Ai Message ==================================

LangGraph was released on January 17, 2024.


In [78]:
# user_input = "I need some expert guidance for building an AI agent. Could you request assistance for me?"
# config = {"configurable": {"thread_id": "1"}}

# events = graph.stream(
#     {"messages": [{"role": "user", "content": user_input}]},
#     config,
#     stream_mode="values",
# )
# for event in events:
#     if "messages" in event:
#         event["messages"][-1].pretty_print()

In [79]:
# snapshot = graph.get_state(config)
# snapshot.next

In [80]:
# human_response = (
#     "We, the experts are here to help! We'd recommend you check out LangGraph to build your agent."
#     " It's much more reliable and extensible than simple autonomous agents."
# )

# human_command = Command(resume={"data": human_response})

# events = graph.stream(human_command, config, stream_mode="values")
# for event in events:
#     if "messages" in event:
#         event["messages"][-1].pretty_print()